Primal problem:
$$\min c^T x \quad\text{subject to}\quad Ax=b, -x\leq 0.$$

Dual problem:
$$\max b^T y \quad\text{subject to}\quad A^Ty\leq c.$$

In [1]:
import numpy as np

In [2]:
# m: number of constraints
# n: number of variables
m = 50
n = 100

A = np.random.randn(m, n)
x0 = np.random.random(n)
b = A.dot(x0)
z = np.random.randn(m)
s = 10*np.random.random(n)

# This is to make sure that the dual problem is feasible in interior since A.T @ z < c.
c = A.T.dot(z) + s

In [3]:
print(x0)

[0.65035492 0.67323186 0.92816192 0.8983248  0.19399717 0.17515678
 0.17698747 0.19413147 0.50102568 0.72094717 0.80311328 0.60901472
 0.57699864 0.42431449 0.92183004 0.51432713 0.04223328 0.90957967
 0.08167692 0.0189576  0.81506351 0.49237968 0.05970939 0.75576439
 0.95132872 0.82619471 0.48542277 0.01314421 0.05607176 0.88051122
 0.36068977 0.62217619 0.87654355 0.80636436 0.91126604 0.32263736
 0.15223093 0.17186548 0.40352169 0.22049532 0.79015087 0.28535272
 0.03601753 0.04335408 0.33829155 0.97531354 0.24501444 0.3270161
 0.6995667  0.53474809 0.38738913 0.11783893 0.91338791 0.23005885
 0.24706215 0.60718321 0.61198209 0.70566917 0.19819126 0.53817014
 0.03773666 0.25438054 0.18346697 0.01069069 0.44342813 0.07124649
 0.22654871 0.36261557 0.90174207 0.3255434  0.40489857 0.25787734
 0.14471045 0.73222662 0.77679616 0.21621091 0.146507   0.48077833
 0.10453449 0.12407576 0.30130278 0.4212824  0.68416816 0.43757223
 0.06133599 0.95307062 0.31791681 0.19146074 0.10719891 0.30277

In [4]:
def get_res(x, lambda_, v, tinv):
    r_dual = Df.dot(lambda_) + A.T.dot(v)+c
    r_cent = -np.diag(lambda_)@f(x) - tinv
    r_pri = A.dot(x)-b
    return r_dual, r_cent, r_pri

def get_res_norm(x, lambda_, v, tinv):
    r_dual, r_cent, r_pri = get_res(x, lambda_, v, tinv)
    return np.linalg.norm(r_dual), np.linalg.norm(r_cent), np.linalg.norm(r_pri)

In [5]:
MAXITERS = 500
TOL = 1e-8
RESTOL = 1e-8
mu = 10
alpha = 0.01
beta = 0.5
gaps = []
resdls = []
x = x0
f = lambda x: -x
Df = -np.eye(n)
lambda_ = -1 / (f(x))
v = np.zeros(m)

for iters in range(MAXITERS):
    # Surrogate duality gap
    ita = - f(x).dot(lambda_)
    gaps.append(ita)
    tinv = ita/(m*mu)

    r_dual, r_cent, r_pri = get_res(x, lambda_, v, tinv)
    resdls.append(np.linalg.norm(np.hstack([r_dual, r_pri])))
    # stopping criterion
    if ita < TOL and np.linalg.norm(np.hstack([r_dual, r_pri])) < RESTOL:
        break
    
    sol = -np.linalg.solve(
        np.block([[np.zeros((n, n)), Df.T, A.T],
                  [-np.diag(lambda_)@Df, -np.diag(f(x)), np.zeros((n, m))],
                  [A, np.zeros((m, m)), np.zeros((m, n))]]),
        np.hstack([r_dual, r_cent, r_pri])
    )
    dx = sol[:n]
    dlambda_ = sol[n:n+n]
    dv = sol[-m:]

    # backtracking line search
    # The maximum step such that the new lambda_ is positive, i.e., feasible.
    step = min(1, 0.99/np.max(-dlambda_/lambda_))
    while True:
        x_new = x + step*dx
        if np.all(f(x_new) < 0):
            break
        step *= beta

    new_x = x + step*dx
    new_lambda_ = lambda_ + step*dlambda_
    new_v = v + step*dv
    
    # Old residual norm
    old_norm = sum(get_res_norm(x, lambda_, v, tinv))

    while sum(get_res_norm(new_x, new_lambda_, new_v, tinv)) > (1- alpha * step) * old_norm:
        step *= beta
        new_x = x + step*dx
        new_lambda_ = lambda_ + step*dlambda_
        new_v = v + step*dv

    x = new_x
    lambda_ = new_lambda_
    v = new_v

print(gaps[-1], resdls[-1], iters)
    

2.329661098059594e-09 3.6722881741864484e-14 28


In [6]:
print(x)

[1.36077222e+00 1.49012773e-02 9.80010908e-01 2.11156505e+00
 4.31982780e-12 1.22786475e+00 4.70934287e-11 1.38224667e-12
 1.24913594e-11 1.12428746e-01 7.19087563e-12 1.37300016e+00
 2.92015627e-12 5.31329969e-12 7.69883127e-03 4.39783710e-12
 4.83275704e-09 1.25979533e-11 1.70621125e-01 3.13462491e-02
 1.35165868e+00 1.37260497e-10 3.35580395e-01 7.36784752e-01
 3.20612670e-12 7.65407723e-01 3.59103782e-12 3.37034922e-12
 5.93664136e-01 6.23517694e-01 1.31863966e+00 6.72457809e-01
 1.38611331e+00 7.67605134e-01 4.68979247e-01 2.71942283e-12
 2.58009075e-12 3.10723439e-12 4.14006018e-01 4.92640571e-01
 5.12291015e-01 1.45615454e-12 2.20710259e-01 4.13909478e-12
 2.94329615e-12 4.54341856e-12 1.03160887e-11 4.52638498e-12
 5.53609148e-01 3.11267474e-12 1.81172966e-01 3.60356403e-12
 1.44194614e+00 7.04849169e-11 5.83487238e-12 2.85946484e-12
 2.35764820e-12 1.42746879e-11 3.23733863e-12 1.75070814e-01
 6.34979784e-12 4.51182087e-01 2.44038504e-12 1.74703000e-12
 3.67901931e-12 1.239821

## Solve by Log-barrier method
In this case, we have the problem
$$\min t c^T x-log(x)\quad \text{subject to}\quad  Ax=b$$

In [7]:
from functions import newton_eq


In [8]:
t = 10
x_inner = x0.copy()
while m/t > TOL:
    f = lambda x: t*c.dot(x) - np.sum(np.log(x))
    grad_f = lambda x: t*c - 1/x
    nabla_f = lambda x: np.diag(1/x**2)

    x,fx_list = newton_eq(f, grad_f, nabla_f, x_inner, A, b, dom_f=lambda x:np.all(x>0), MAXITERS=50, TOL=1e-5, alpha=0.001, beta=0.8)
    x_inner = x[-1]
    t *= mu

In [9]:
x_inner

array([1.36078728e+00, 1.48960559e-02, 9.80018487e-01, 2.11154631e+00,
       1.85426935e-10, 1.22787330e+00, 2.02146742e-09, 5.93324028e-11,
       5.36186916e-10, 1.12420125e-01, 3.08665590e-10, 1.37299697e+00,
       1.25346579e-10, 2.28071333e-10, 7.69382594e-03, 1.88775475e-10,
       2.07429199e-07, 5.40762418e-10, 1.70606826e-01, 3.13431358e-02,
       1.35165597e+00, 5.89185298e-09, 3.35565701e-01, 7.36816705e-01,
       1.37621738e-10, 7.65399135e-01, 1.54143902e-10, 1.44670943e-10,
       5.93645270e-01, 6.23501321e-01, 1.31865438e+00, 6.72460571e-01,
       1.38610312e+00, 7.67614930e-01, 4.68980860e-01, 1.16730172e-10,
       1.10749394e-10, 1.33376830e-10, 4.14012279e-01, 4.92633527e-01,
       5.12279733e-01, 6.25048675e-11, 2.20718253e-01, 1.77669034e-10,
       1.26339847e-10, 1.95024457e-10, 4.42814080e-10, 1.94293299e-10,
       5.53599732e-01, 1.33610376e-10, 1.81168619e-01, 1.54681585e-10,
       1.44194001e+00, 3.02553705e-09, 2.50459660e-10, 1.22741429e-10,
      

We can see that this is not stable. We need to choose alpha carefully, otherwise it will not converge. The primal-dual method is more stable than barrier method.